# Batch Predictions

In [57]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [58]:
!pip install --upgrade sagemaker==1.56.1

Requirement already up-to-date: sagemaker==1.56.1 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (1.56.1)


In [59]:
!pip list

Package                            Version
---------------------------------- ----------
absl-py                            0.9.0
alabaster                          0.7.10
anaconda-client                    1.6.14
anaconda-project                   0.8.2
asn1crypto                         0.24.0
astor                              0.8.1
astroid                            1.6.3
astropy                            3.0.2
attrs                              18.1.0
Automat                            0.3.0
autovizwidget                      0.15.0
awscli                             1.18.49
Babel                              2.5.3
backcall                           0.1.0
backports.shutil-get-terminal-size 1.0.0
bcrypt                             3.1.7
beautifulsoup4                     4.6.0
bitarray                           0.8.1
bkcharts                           0.2
blaze                              0.11.3
bleach                             2.1.3
bokeh                              1.0.4
bot

zipp                               3.0.0


# Setup Batch Transform Model

In [60]:
%store -r training_job_name

In [61]:
print(training_job_name)

tensorflow-training-2020-04-30-03-09-12-331


In [62]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-835319576252/tensorflow-training-2020-04-30-03-09-12-331/output/model.tar.gz to ./model.tar.gz


In [63]:
!tar -xvzf ./model.tar.gz

tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/assets/
transformers/
transformers/fine-tuned/
transformers/fine-tuned/config.json
transformers/fine-tuned/tf_model.h5


In [64]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

2020-05-06 02:52:56.449285: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:
2020-05-06 02:52:56.449423: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/ef

In [65]:
!pygmentize ./src_csv/inference.py

import json
import tensorflow as tf
from transformers import DistilBertTokenizer

classes=[1, 2, 3, 4, 5]
max_seq_length=128
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def input_handler(data, context):
    transformed_instances = []
    print(type(data))
    print(data)

    for instance in data:
        print(type(instance))
        print(instance)

        data_str = instance.decode('utf-8')
        print(type(data_str))
        print(data_str)

        data_str_split = data_str.split('\t')
        print(len(data_str_split))
        if (len(data_str_split) >= 13):
            print(data_str_split[13])

        tokens_a = tokenizer.tokenize(data_str_split[13])

        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

        tokens = []  
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
    

In [77]:
from sagemaker.tensorflow.serving import Model

# If you change SAGEMAKER_TFS_DEFAULT_MODEL_NAME to something other than 'saved_model', you may see the dreaded ping error in the logs error
batch_env = {
  'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'saved_model', # <== change this when using multi-model,
                                                     #     but watch out for the dreaded ping/ error 
                                                     #     if the model name doesn't exist
  'SAGEMAKER_TFS_ENABLE_BATCHING': 'true',
  'SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS': '50000',
  'SAGEMAKER_TFS_MAX_BATCH_SIZE': '100000'
}

batch_model = Model(entry_point='inference.py', # <== This cannot change. See https://github.com/aws/sagemaker-python-sdk/issues/1451
                    source_dir='src_csv',       
                    model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
                    role=role,
                    framework_version='2.1.0',
                    env=batch_env)

In [78]:
batch_predictor = batch_model.transformer(instance_count=1,
                                          strategy='MultiRecord', 
                                          instance_type='ml.c5.18xlarge',
                                          accept='text/csv',
                                          assemble_with='Line',
                                          max_payload=100, # This is in Megabytes (not number of records)
                                          env=batch_env)

# Specify Input Data

In [79]:
predict_csv_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)

# Start Batch Predictions

In [80]:
batch_predictor.transform(data=predict_csv_s3_uri,
                          split_type='Line',
                          compression_type='Gzip',
                          content_type='text/csv',
                          experiment_config=None,
                          wait=False)

In [81]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/{}?region={}&tab=Monitor">Batch Prediction Job</a></b>'.format(region, batch_predictor.latest_transform_job.job_name, region)))


In [82]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TransformJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'.format(region, batch_predictor.latest_transform_job.job_name)))


In [83]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/s3/buckets/{}/{}/?region={}">Batch Prediction S3 Output</a></b>'.format(bucket, batch_predictor.latest_transform_job.job_name, region)))


In [84]:
print('Waiting for batch prediction job: ' + batch_predictor.latest_transform_job.job_name)

batch_predictor.wait(logs=False)

Waiting for batch prediction job: tensorflow-inference-2020-05-06-03-21-3-2020-05-06-03-21-33-861
...................................................................*


UnexpectedStatusException: Error for Transform job tensorflow-inference-2020-05-06-03-21-3-2020-05-06-03-21-33-861: Failed. Reason: AlgorithmError: See job logs for more information

# Wait Until the ^^ Batch Transform ^^ Completes

# Check Output Data

After the transform job has completed, download the output data from S3.

For each file in the input data, we have a corresponding file with a ".out" extension.  This .out file contains the predicted labels for each input row. 

In [ ]:
# Download the output data from S3 to local filesystem
batch_prediction_output_s3_uri = batch_predictor.output_path

In [ ]:
%%bash 

aws s3 cp --recursive $batch_prediction_output_s3_uri/ ./batch_prediction_output

ls ./batch_prediction_output